In [1]:
import numpy as np
def check_input_data(data):
    return_bool=False
    if not "temperature" in data:
        print("'temperature' key does not exists in dict") 
        return_bool=True
    if not "pressure" in data:
        print("'pressure' key does not exists in dict") 
        return_bool=True
    if not "api" in data:
        print("'api' key does not exists in dict")
        return_bool=True
    if not "gas_oil_ratio" in data:
        print("'gas_oil_ratio' key does not exists in dict")
        return_bool=True
    if not "gas_gravity" in data:
        print("'gas_gravity' key does not exists in dict")
        return_bool=True
    if not "water_salinity" in data:
        print("'water_salinity' key does not exists in dict")
        return_bool=True
    return return_bool
    
def glaso_pb(data):
    pb=1.0e20
    if data['gas_oil_ratio'] < 20000.1:
        l_param=np.log10(np.power((data['gas_oil_ratio']/data['gas_gravity']),0.816)*np.power(data['temperature'],0.172)/np.power(data['api'],0.989))
        pb=np.power(10.0,1.7669+1.7447*l_param-0.30218*l_param*l_param)
    return pb
def glaso_rs(data):
    rs=data['gas_oil_ratio']
    if data['pressure']<glaso_pb(data):
        l_v=14.1811-3.3093*np.log10(data['pressure'])
        if l_v < 0.0:
            l_v=0.0
        l_a=np.power(10.0,2.8869-np.sqrt(l_v))
        rs=data['gas_gravity']*np.power(l_a*np.power(data['api'],0.989)/np.power(data['temperature'],0.172),1.2255) 
    return rs
def glaso_bo(data,rs):
    grvo=141.5/(data['api']+131.5)
    l_temp=np.log10(rs*np.power(data['gas_gravity']/grvo,0.526)+0.968*data['temperature'])
    bo=1.0+np.power(10.0,-6.58511+2.91329*l_temp-0.27683*l_temp*l_temp)
    return bo
def beggs_oil_viscosity(p,t,api,rs,pb):
    rs1=min(3000.0,rs)
    a=10.715*np.power(rs1+100.0,-0.515)
    b=5.44*np.power(rs1+150,-0.338)
    g=np.power(10.0,3.0324-0.02023*api)
    h=min(1.5,g*np.power(t,-1.163))
    u=np.power(10.0,h)-1.0
    mu_oil=a*np.power(u,b)
    if p>pb:
        e=np.exp((-8.98*np.power(10.0,-5.0))*p-11.513)
        d=2.6*np.power(p,1.187)*e
        mu_oil*=np.power(p/pb,d)
    return mu_oil
def glaso_correlation(data):
    pb=glaso_pb(data)
    rs=glaso_rs(data)
    rs=min(data['gas_oil_ratio'],rs)
    rs1=min(10000.0,rs)
    co=(-1433.0+5.0*rs1+17.2*data['temperature']-1180.0*data['gas_gravity']+12.61*data['api'])/(data['pressure']*1.0e05)
    co=max(co,1.0e-06)
    bo=glaso_bo(data,min(20000.0,rs))
    if data['pressure']>pb:
        bo=bo*(np.exp(co*(pb-data['pressure']))-1.0)+bo*(1.0-co*(data['pressure']-pb))
    return {'bubble_point':pb,'solution_gas':rs,'compressibility':co,'fvf':bo}
def oil_properties(data):
    if check_input_data(data):
        return {'bubble_point':-1.0,'solution_gas':-1.0,'compressibility':-1.0,'fvf':-1.0,'viscosity':-1.0}
    oil=glaso_correlation(data)
    oil['viscosity']=beggs_oil_viscosity(data['pressure'],
                                         data['temperature'],
                                         data['api'],
                                         oil['solution_gas'],
                                         oil['bubble_point'])
    return oil
    

In [2]:
# testing
test_input={'api':35.0,
            'gas_oil_ratio':500.0,
            'gas_gravity':0.72,
            'pressure':3200.0,
            'temperature':185.0,
            'water_salinity':35000.0}

#print(oil_properties(test_input))

In [3]:
def water_properties(data):
    if check_input_data(data):
        return {'compressibility':-1.0,'fvf':-1.0,'viscosity':-1.0}
    dis=data['water_salinity']/10000.0
    ar=0.14
    br=0.58
    if data['temperature']>=150.0:
        ar-=0.0008*(data['temperature']-150.0)
        br+=0.0011*(data['temperature']-150.0)
    fvf=1.0
    if data['temperature']>40.0:
        fvf=0.998+4.1912e-06*np.power(data['temperature']-32.0,1.7827)
    cw=(3.1e-06-8.0e-11*data['pressure'])+1.69e-10*np.power(abs(data['temperature']-120.0),1.674)
    cw=max(1.0e-10,cw)
    dvwdpr=2.91e-06+7.575e-11*np.power(abs(data['temperature']-115.0),1.859)
    fvf=fvf-dvwdpr*data['pressure']
    mu_water=np.power(176.0/(data['temperature']+100.0),2.11)
    mu_slope=0.018+1.2e-05*abs(150.0-data['temperature'])
    mu_water *= (1.0+mu_slope*dis)
    return {'compressibility':cw,'fvf':fvf,'viscosity':mu_water}

In [4]:
#print(water_properties(test_input))

In [5]:
def gas_pc_tc(sg):
    tci=168.0+(325.0*sg)-(12.5*sg*sg)
    pci=706.0-(51.7*sg)-(11.1*sg*sg)
    return tci,pci

def zfactor(t,tc,p,pc):
    Niter=50
    val1=1.0
    A=[0.31506237,
       -1.04670990,
       -0.57832729,
       0.53530771,
       -0.61232032,
       -0.10488813,
       0.68157001,
       0.68446549]
    tr=(t+459.67)/tc
    pr=p/pc
    t1=A[0]*tr+A[1]+A[2]/(tr*tr)
    t2=A[3]*tr+A[4]
    t3=A[4]*A[5]
    t4=A[6]/(tr*tr)
    t5=A[7]
    drd=1.0
    drd1=1.1 #start with a difference
    xdft=1.e20
    i=0
    while abs(drd-drd1)>0.000001 and i<Niter:
        i+=1
        drd2=drd*drd
        drd3=drd2*drd
        drd4=drd2*drd2
        drd5=drd3*drd2
        p_calc=(tr + t1*drd + t2*drd2 + t3*drd5)*drd+ t4*drd3*(1.0 + t5*drd2)*np.exp(-t5*drd2)
        dp_calc=tr + 2.0*t1*drd + 3.0*t2*drd2 + 6.0*t3*drd5+ t4*drd2*np.exp(-t5*drd2)*(3.0 + 3.0*t5*drd2- 2.0*t5*t5*drd4)
        dp_calc=abs(dp_calc)*val1
        if dp_calc==0.0:
            dp_calc=1.0
        drd1=drd-(p_calc-0.270*pr)/dp_calc
        if drd1 <= 0.0:
            drd1=0.25*drd
        if drd1 > 5.2:
            drd1=drd+0.9*(5.2-drd)
        if abs(drd-drd1)<xdft:
            xdft=abs(drd-drd1)
        drdsav=drd1
        drd=(drd+drd1)*0.5
    zed=max(0.270*pr/(drdsav*tr),1.0e-06)
    return zed 

def gas_viscosity(t,p,g,z):
    MW=28.966*g
    TR=459.67+t
    R1 = 669.8
    P1 = (9.4 +0.02*MW)* np.power(TR,1.5)
    P2 = (209.0 + (19.0*MW) + TR) * 1.0e4
    A1 = P1 / P2
    B1 = 3.5 + (986.0/TR) + (0.01*MW)
    C1 = 2.4 - (0.2*B1)
    P3 = p * MW
    P4 = z * R1 * TR
    DN = P3 / P4
    return A1*np.exp(B1*np.power(DN,C1))
    
def gas_properties(data):
    if check_input_data(data):
        return {'compressibility':-1.0,'fvf':-1.0,'viscosity':-1.0,'z_factor':-1.0}
    tc,pc=gas_pc_tc(data['gas_gravity'])
    z=zfactor(data['temperature'],tc,data['pressure'],pc)
    tr=data['temperature']+459.67
    fvf=2.8301e-02*z*tr/data['pressure']
    delta=5.0
    z_delta=zfactor(data['temperature'],tc,data['pressure']+delta,pc)
    fvf2=2.8301e-02*z_delta*tr/data['pressure']
    dbgdp=(fvf-fvf2)/delta
    cg=abs((1.0/fvf)*dbgdp)
    mug=gas_viscosity(data['temperature'],data['pressure'],data['gas_gravity'],z)
    # reporting gas fvf in units of rb/scf for ease of calculation elsewhere...
    return {'compressibility':cg,'fvf':fvf/5.615,'viscosity':mug,'z_factor':z}
    

In [6]:
#print(gas_properties(test_input))

In [7]:
def formation_compressibility(porosity):
    # SPE SPE 953309-G Hall 1953 shows a plot of porosity vs Cf.
    # The reason we use the fit below is that it ensures that the derivative of the Cf is zero when 
	# phi = 30% so we do not have a discontinuity in the derivative at phi = 30%.
	# This does mean that we diverge from Hall's plot at por < 4% but we are unlikely to be looking
	# at cases with such low porosity.
    cf=3.2e-06
    if porosity<0.3:
        cf+=np.power(0.3-porosity,2.415)*7.8e-05
    return cf